In [1]:
import pandas as pd
import numpy as np
import math

In [2]:
chess = pd.read_csv('../data/caissabase_df.csv', index_col = 0)

C:\Users\apcsm\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (9,12,15,16) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\apcsm\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
chess.loc[chess['result'] == '1-0', 'white_perf'] = chess['black_elo'] + 400 
chess.loc[chess['result'] == '1/2-1/2', 'white_perf'] = chess['black_elo']
chess.loc[chess['result'] == '0-1', 'white_perf'] = chess['black_elo'] - 400

In [4]:
chess.loc[chess['result'] == '1-0', 'black_perf'] = chess['white_elo'] - 400 
chess.loc[chess['result'] == '1/2-1/2', 'black_perf'] = chess['white_elo']
chess.loc[chess['result'] == '0-1', 'black_perf'] = chess['white_elo'] + 400

In [5]:
chess['white_perf_diff'] = chess['white_perf']-chess['white_elo']
chess['black_perf_diff'] = chess['black_perf']-chess['black_elo']

In [6]:
white_first_move_adv = chess['white_perf_diff'].mean()

In [7]:
chess['white_perf_diff_adjusted'] = chess['white_perf_diff']-white_first_move_adv
chess['black_perf_diff_adjusted'] = chess['black_perf_diff']+white_first_move_adv

In [8]:
chess = chess.astype({'white_perf':'int', 'black_perf':'int', 'white_perf_diff':'int', 'black_perf_diff':'int', 'white_perf_diff_adjusted':'int', 'black_perf_diff_adjusted':'int'})

In [9]:
chess

,event,white_elo,black_elo,result,site,date,white,white_fide_id,white_title,black,...,19w,19b,20w,20b,white_perf,black_perf,white_perf_diff,black_perf_diff,white_perf_diff_adjusted,black_perf_diff_adjusted
0,Tel Aviv Int. 2021,2337,2251,1/2-1/2,Tel Aviv ISR,2021-11-29,"Solomon, Asif",2820218.0,FM,"Kaganskiy, G",...,Bd1,Nc5,b4,Nd3,2251,2337,-86,86,-117,117
1,ch-NED Final 2021,2608,2450,1-0,Rotterdam NED,2021-11-29,"Warmerdam, Max",1048104.0,GM,"Lai, Hing Ting",...,Qd3,g6,Nd2,Kg7,2850,2208,242,-242,210,-210
2,Tel Aviv Int. 2021,2299,2517,1/2-1/2,Tel Aviv ISR,2021-11-29,"Aizenberg, Benny",2821125.0,FM,"Rozentalis, E",...,bxa5,h5,Bxf4,exf4,2517,2299,218,-218,186,-186
3,Tel Aviv Int. 2021,2412,2297,1/2-1/2,Tel Aviv ISR,2021-11-29,"Kantsler, B",2805359.0,GM,"Zemach, Roee",...,Bxd5,Na5,Bxb7,Nxb7,2297,2412,-115,115,-146,146
4,ch-NED Final 2021,2570,2523,1/2-1/2,Rotterdam NED,2021-11-29,"Pruijssers, R",1013068.0,GM,"Swinkels, R",...,a4,Rh5,Kb2,Rah8,2523,2570,-47,47,-78,78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1850114,Capodanno,2373,2544,0-1,Reggio Emilia ITA,2000-01-01,"Manca, Federico",NaN,NaN,"Solozhenkin, Evgeniy",...,Ne4,Rd5,Bc3,Qd8,2144,2773,-229,229,-260,260
1850115,Hilton Open,2131,2458,0-1,Basel SUI,2000-01-01,"Speck, Hans",NaN,NaN,"Berzinsh, Roland",...,Qc2,Bg4,h4,Qh6,2058,2531,-73,73,-104,104
1850116,Hambourg IHEM,2258,2339,1-0,Hambourg,2000-01-01,"Voigt, Martin",NaN,NaN,"Lindinger, Markus",...,Nd4,b4,cxb4,axb4,2739,1858,481,-481,449,-449
1850117,Hilton Open,2565,2330,1-0,Basel SUI,2000-01-01,"Sokolov, An",NaN,NaN,"Milosevic, G.",...,NaN,NaN,NaN,NaN,2730,2165,165,-165,133,-133


In [23]:
w1 = chess.groupby(['1w']).agg(mean = ('white_perf_diff_adjusted', 'mean'), count = ('white_perf_diff_adjusted', 'count'), standard_deviation = ('white_perf_diff_adjusted', 'std'))
w1['95%_confidence_interval_+'] = w1['mean'] + 1.96*(w1['standard_deviation']/(np.sqrt(w1['count'])))
w1['95%_confidence_interval_-'] = w1['mean'] - 1.96*(w1['standard_deviation']/(np.sqrt(w1['count'])))
w1['+/-range'] = (w1['95%_confidence_interval_+']-w1['95%_confidence_interval_-'])/2
w1_count = w1['count'].sum()
w1['%_of_previous_position'] = w1['count']/w1_count
w1 = w1.reset_index()
w1.loc[w1['count'] >= 25, 'countscore'] = 50*(w1['count'])**-0.5
w1.loc[w1['count'] >= 25, 'eloscore'] = (0.08*w1['mean'])
w1.loc[w1['eloscore'] > 10, 'eloscore'] = 10
w1.loc[w1['count'] >= 25, 'rarityscore'] = ((w1['%_of_previous_position']**-1)-2)/2
w1.loc[w1['rarityscore'] < 0, 'rarityscore'] = 0
w1.loc[w1['rarityscore'] > 10, 'rarityscore'] = 10
w1['score'] = w1['countscore']+w1['eloscore']+w1['rarityscore']
w1.sort_values(by = 'score', ascending = False).reset_index()

,index,1w,mean,count,standard_deviation,95%_confidence_interval_+,95%_confidence_interval_-,+/-range,%_of_previous_position,countscore,eloscore,rarityscore,score
0,18,h3,-7.839286,56,339.659229,81.122907,-96.801478,88.962192,0.000030,6.681531,-0.627143,10.000000,16.054388
1,19,h4,-78.051282,39,317.639563,21.640270,-177.742834,99.691552,0.000021,8.006408,-6.244103,10.000000,11.762305
2,3,Nh3,-86.897436,39,251.558384,-7.945540,-165.849332,78.951896,0.000021,8.006408,-6.951795,10.000000,11.054613
3,8,c3,-32.503247,308,296.761604,0.639494,-65.645988,33.142741,0.000166,2.849014,-2.600260,10.000000,10.248755
4,4,a3,-27.474085,656,306.116741,-4.048458,-50.899713,23.425627,0.000355,1.952172,-2.197927,10.000000,9.754245
5,10,d3,-30.037152,646,297.939157,-7.061523,-53.012780,22.975628,0.000349,1.967224,-2.402972,10.000000,9.564252
6,1,Nc3,-19.921842,2866,292.389007,-9.217031,-30.626654,10.704811,0.001549,0.933968,-1.593747,10.000000,9.340220
7,16,g3,-18.765801,13433,280.334615,-14.025059,-23.506542,4.740742,0.007261,0.431403,-1.501264,10.000000,8.930139
8,6,b3,-22.657389,8946,288.830136,-16.672114,-28.642664,5.985275,0.004835,0.528635,-1.812591,10.000000,8.716043
9,7,b4,-34.859571,1353,305.866689,-18.561382,-51.157761,16.298189,0.000731,1.359318,-2.788766,10.000000,8.570552


In [24]:
b1 = chess.groupby(['1w','1b']).agg(mean = ('white_perf_diff_adjusted', 'mean'), count = ('white_perf_diff_adjusted', 'count'), standard_deviation = ('white_perf_diff_adjusted', 'std'))
b1['95%_confidence_interval_+'] = b1['mean'] + 1.96*(b1['standard_deviation']/(np.sqrt(b1['count'])))
b1['95%_confidence_interval_-'] = b1['mean'] - 1.96*(b1['standard_deviation']/(np.sqrt(b1['count'])))
b1['+/-range'] = (b1['95%_confidence_interval_+']-b1['95%_confidence_interval_-'])/2
b1 = b1.reset_index()
b1p = b1.groupby(['1w']).agg(previous_count = ('count','sum'))
b1 = b1.merge(b1p, how = 'outer', on = '1w')
b1['%_of_previous_position'] = b1['count'] / b1['previous_count']
b1.loc[b1['count'] >= 25, 'countscore'] = 50*(b1['count'])**-0.5
b1.loc[b1['count'] >= 25, 'eloscore'] = (-0.08*b1['mean'])
b1.loc[b1['eloscore'] > 10, 'eloscore'] = 10
b1.loc[b1['count'] >= 25, 'rarityscore'] = ((b1['%_of_previous_position']**-1)-2)/2
b1.loc[b1['rarityscore'] < 0, 'rarityscore'] = 0
b1.loc[b1['rarityscore'] > 10, 'rarityscore'] = 10
b1['score'] = b1['countscore']+b1['eloscore']+b1['rarityscore']
b1.sort_values(by = 'score', ascending = False).reset_index()

,index,1w,1b,mean,count,standard_deviation,95%_confidence_interval_+,95%_confidence_interval_-,+/-range,previous_count,%_of_previous_position,countscore,eloscore,rarityscore,score
0,210,f4,f5,-102.142857,49,277.489264,-24.445863,-179.839851,77.696994,5951,0.008234,7.142857,8.171429,10.000000,25.314286
1,201,f4,Nh6,-94.386364,44,234.097298,-25.215032,-163.557695,69.171331,5951,0.007394,7.537784,7.550909,10.000000,25.088693
2,85,b4,a5,-78.939394,33,321.999800,30.924428,-188.803216,109.863822,1353,0.024390,8.703883,6.315152,10.000000,25.019034
3,10,Nc3,c6,-27.961538,26,274.019632,77.368217,-133.291294,105.329755,2866,0.009072,9.805807,2.236923,10.000000,22.042730
4,214,g3,Nc6,-27.766667,30,315.328584,85.072218,-140.605551,112.838885,13433,0.002233,9.128709,2.221333,10.000000,21.350043
5,123,c4,g5,-12.500000,30,296.396839,93.564247,-118.564247,106.064247,122899,0.000244,9.128709,1.000000,10.000000,20.128709
6,50,a3,c5,-3.448276,29,281.855914,99.136809,-106.033361,102.585085,656,0.044207,9.284767,0.275862,10.000000,19.560629
7,77,b3,e6,-42.557143,70,305.774978,29.075174,-114.189459,71.632316,8946,0.007825,5.976143,3.404571,10.000000,19.380714
8,203,f4,b6,-44.857143,77,265.309041,14.403009,-104.117295,59.260152,5951,0.012939,5.698029,3.588571,10.000000,19.286600
9,75,b3,d6,-68.983516,182,283.670622,-27.770468,-110.196565,41.213048,8946,0.020344,3.706247,5.518681,10.000000,19.224928


In [25]:
w2 = chess.groupby(['1w','1b','2w']).agg(mean = ('white_perf_diff_adjusted', 'mean'), count = ('white_perf_diff_adjusted', 'count'), standard_deviation = ('white_perf_diff_adjusted', 'std'))
w2['95%_confidence_interval_+'] = w2['mean'] + 1.96*(w2['standard_deviation']/(np.sqrt(w2['count'])))
w2['95%_confidence_interval_-'] = w2['mean'] - 1.96*(w2['standard_deviation']/(np.sqrt(w2['count'])))
w2['+/-range'] = (w2['95%_confidence_interval_+']-w2['95%_confidence_interval_-'])/2
w2 = w2.reset_index()
w2p = w2.groupby(['1w','1b']).agg(previous_count = ('count','sum'))
w2 = w2.merge(w2p, how = 'outer', on = ['1w','1b'])
w2['%_of_previous_position'] = w2['count'] / w2['previous_count']
w2.loc[w2['count'] >= 25, 'countscore'] = 50*(w2['count'])**-0.5
w2.loc[w2['count'] >= 25, 'eloscore'] = (0.08*w2['mean'])
w2.loc[w2['eloscore'] > 10, 'eloscore'] = 10
w2.loc[w2['count'] >= 25, 'rarityscore'] = ((w2['%_of_previous_position']**-1)-2)/2
w2.loc[w2['rarityscore'] < 0, 'rarityscore'] = 0
w2.loc[w2['rarityscore'] > 10, 'rarityscore'] = 10
w2['score'] = w2['countscore']+w2['eloscore']+w2['rarityscore']
w2.sort_values(by = 'score', ascending = False).reset_index()

,index,1w,1b,2w,mean,count,standard_deviation,95%_confidence_interval_+,95%_confidence_interval_-,+/-range,previous_count,%_of_previous_position,countscore,eloscore,rarityscore,score
0,1036,e4,b6,c4,203.666667,27,245.147365,296.136786,111.196547,92.470120,1897,0.014233,9.622504,10.000000,10.000000,29.622504
1,201,Nf3,f5,b4,113.184211,38,267.178537,198.134691,28.233730,84.950481,4848,0.007838,8.111071,9.054737,10.000000,27.165808
2,1304,g3,d5,d3,61.600000,35,219.786019,134.415211,-11.215211,72.815211,4851,0.007215,8.451543,4.928000,10.000000,23.379543
3,988,e4,Nc6,Bb5,37.500000,32,283.332929,135.669851,-60.669851,98.169851,4709,0.006795,8.838835,3.000000,10.000000,21.838835
4,1340,g3,g6,c4,38.318182,44,262.978801,116.023452,-39.387088,77.705270,1894,0.023231,7.537784,3.065455,10.000000,20.603238
5,543,c4,c5,e3,10.312500,32,273.903378,105.215179,-84.590179,94.902679,14815,0.002160,8.838835,0.825000,10.000000,19.663835
6,197,Nf3,e6,e4,43.941176,68,269.036861,107.887179,-20.004826,63.946003,3176,0.021411,6.063391,3.515294,10.000000,19.578685
7,1275,g3,Nf6,Nf3,18.289474,38,308.711088,116.445394,-79.866447,98.155921,2680,0.014179,8.111071,1.463158,10.000000,19.574229
8,1109,e4,d6,d3,28.200000,50,290.128191,108.619432,-52.219432,80.419432,36265,0.001379,7.071068,2.256000,10.000000,19.327068
9,1031,e4,b6,Nf3,2.516129,31,295.500014,106.539933,-101.507675,104.023804,1897,0.016342,8.980265,0.201290,10.000000,19.181555


In [43]:
b2 = chess.groupby(['1w','1b','2w','2b']).agg(mean = ('white_perf_diff_adjusted', 'mean'), count = ('white_perf_diff_adjusted', 'count'), standard_deviation = ('white_perf_diff_adjusted', 'std'))
b2['95%_confidence_interval_+'] = b2['mean'] + 1.96*(b2['standard_deviation']/(np.sqrt(b2['count'])))
b2['95%_confidence_interval_-'] = b2['mean'] - 1.96*(b2['standard_deviation']/(np.sqrt(b2['count'])))
b2['+/-range'] = (b2['95%_confidence_interval_+']-b2['95%_confidence_interval_-'])/2
b2 = b2.reset_index()
b2p = b2.groupby(['1w','1b','2w']).agg(previous_count = ('count','sum'))
b2 = b2.merge(b2p, how = 'outer', on = ['1w','1b','2w'])
b2['%_of_previous_position'] = b2['count'] / b2['previous_count']
b2.loc[b2['count'] >= 25, 'countscore'] = 50*(b2['count'])**-0.5
b2.loc[b2['count'] >= 25, 'eloscore'] = (-0.08*b2['mean'])
b2.loc[b2['eloscore'] > 10, 'eloscore'] = 10
b2.loc[b2['count'] >= 25, 'rarityscore'] = ((b2['%_of_previous_position']**-1)-2)/2
b2.loc[b2['rarityscore'] < 0, 'rarityscore'] = 0
b2.loc[b2['rarityscore'] > 10, 'rarityscore'] = 10
b2['score'] = b2['countscore']+b2['eloscore']+b2['rarityscore']
b2.sort_values(by = 'score', ascending = False).reset_index()

,index,1w,1b,2w,2b,mean,count,standard_deviation,95%_confidence_interval_+,95%_confidence_interval_-,+/-range,previous_count,%_of_previous_position,countscore,eloscore,rarityscore,score
0,4344,g3,c5,Bg2,d5,-117.485714,35,242.834532,-37.034520,-197.936908,80.451194,1163,0.030095,8.451543,9.398857,10.000000,27.850400
1,3558,e4,c5,g3,d6,-119.025000,40,325.955439,-18.010363,-220.039637,101.014637,971,0.041195,7.905694,9.522000,10.000000,27.427694
2,1956,c4,g6,Nf3,Nf6,-114.172414,29,288.324726,-9.232921,-219.111907,104.939493,567,0.051146,9.284767,9.133793,8.775862,27.194422
3,4343,g3,c5,Bg2,Nf6,-130.362069,58,311.883171,-50.095626,-210.628512,80.266443,1163,0.049871,6.565322,10.000000,9.025862,25.591184
4,3478,e4,c5,a3,d6,-79.758621,29,285.109334,24.010589,-183.527830,103.769210,572,0.050699,9.284767,6.380690,8.862069,24.527526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4666,4666,h4,e5,h5,Nc6,-85.666667,3,494.297818,473.683971,-645.017305,559.350638,6,0.500000,NaN,NaN,NaN,NaN
4667,4667,h4,e5,h5,d5,303.000000,2,137.178716,493.120000,112.880000,190.120000,6,0.333333,NaN,NaN,NaN,NaN
4668,4668,h4,e6,h5,c5,-36.000000,1,NaN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN
4669,4669,h4,h5,Nf3,g6,-117.000000,1,NaN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN


In [44]:
w3 = chess.groupby(['1w','1b','2w','2b','3w']).agg(mean = ('white_perf_diff_adjusted', 'mean'), count = ('white_perf_diff_adjusted', 'count'), standard_deviation = ('white_perf_diff_adjusted', 'std'))
w3['95%_confidence_interval_+'] = w3['mean'] + 1.96*(w3['standard_deviation']/(np.sqrt(w3['count'])))
w3['95%_confidence_interval_-'] = w3['mean'] - 1.96*(w3['standard_deviation']/(np.sqrt(w3['count'])))
w3['+/-range'] = (w3['95%_confidence_interval_+']-w3['95%_confidence_interval_-'])/2
w3 = w3.reset_index()
w3p = w3.groupby(['1w','1b','2w','2b']).agg(previous_count = ('count','sum'))
w3 = w3.merge(w3p, how = 'outer', on = ['1w','1b','2w','2b'])
w3['%_of_previous_position'] = w3['count'] / w3['previous_count']
w3.loc[w3['count'] >= 25, 'countscore'] = 50*(w3['count'])**-0.5
w3.loc[w3['count'] >= 25, 'eloscore'] = (0.08*w3['mean'])
w3.loc[w3['eloscore'] > 10, 'eloscore'] = 10
w3.loc[w3['count'] >= 25, 'rarityscore'] = ((w3['%_of_previous_position']**-1)-2)/2
w3.loc[w3['rarityscore'] < 0, 'rarityscore'] = 0
w3.loc[w3['rarityscore'] > 10, 'rarityscore'] = 10
w3['score'] = w3['countscore']+w3['eloscore']+w3['rarityscore']
w3.sort_values(by = 'score', ascending = False).reset_index()

,index,1w,1b,2w,2b,3w,mean,count,standard_deviation,95%_confidence_interval_+,95%_confidence_interval_-,+/-range,previous_count,%_of_previous_position,countscore,eloscore,rarityscore,score
0,10773,e4,e6,d3,c5,Nc3,118.925926,27,320.905201,239.972066,-2.120214,121.046140,1393,0.019383,9.622504,9.514074,10.0,29.136579
1,6530,d4,d5,Bg5,h6,Bf4,94.269231,26,281.643826,202.529632,-13.991170,108.260401,743,0.034993,9.805807,7.541538,10.0,27.347345
2,8523,e4,Nc6,Nf3,d6,Bb5,90.653846,26,291.967114,202.882391,-21.574699,112.228545,1406,0.018492,9.805807,7.252308,10.0,27.058114
3,11082,e4,g6,d4,d6,c4,124.888889,54,271.864538,197.401201,52.376577,72.512312,1270,0.042520,6.804138,9.991111,10.0,26.795249
4,7541,d4,e6,e4,d5,Bd3,113.446809,47,302.235857,199.854645,27.038972,86.407836,2170,0.021659,7.293250,9.075745,10.0,26.368994
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12447,12447,h4,e5,h5,Nc6,c3,-65.000000,2,697.207286,901.280000,-1031.280000,966.280000,3,0.666667,NaN,NaN,NaN,NaN
12448,12448,h4,e5,h5,d5,c3,303.000000,2,137.178716,493.120000,112.880000,190.120000,2,1.000000,NaN,NaN,NaN,NaN
12449,12449,h4,e6,h5,c5,c3,-36.000000,1,NaN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN
12450,12450,h4,h5,Nf3,g6,Ng5,-117.000000,1,NaN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN


In [45]:
b3 = chess.groupby(['1w','1b','2w','2b','3w','3b']).agg(mean = ('white_perf_diff_adjusted', 'mean'), count = ('white_perf_diff_adjusted', 'count'), standard_deviation = ('white_perf_diff_adjusted', 'std'))
b3['95%_confidence_interval_+'] = b3['mean'] + 1.96*(b3['standard_deviation']/(np.sqrt(b3['count'])))
b3['95%_confidence_interval_-'] = b3['mean'] - 1.96*(b3['standard_deviation']/(np.sqrt(b3['count'])))
b3['+/-range'] = (b3['95%_confidence_interval_+']-b3['95%_confidence_interval_-'])/2
b3 = b3.reset_index()
b3p = b3.groupby(['1w','1b','2w','2b','3w']).agg(previous_count = ('count','sum'))
b3 = b3.merge(b3p, how = 'outer', on = ['1w','1b','2w','2b','3w'])
b3['%_of_previous_position'] = b3['count'] / b3['previous_count']
b3.loc[b3['count'] >= 25, 'countscore'] = 50*(b3['count'])**-0.5
b3.loc[b3['count'] >= 25, 'eloscore'] = (-0.08*b3['mean'])
b3.loc[b3['eloscore'] > 10, 'eloscore'] = 10
b3.loc[b3['count'] >= 25, 'rarityscore'] = ((b3['%_of_previous_position']**-1)-2)/2
b3.loc[b3['rarityscore'] < 0, 'rarityscore'] = 0
b3.loc[b3['rarityscore'] > 10, 'rarityscore'] = 10
b3['score'] = b3['countscore']+b3['eloscore']+b3['rarityscore']
b3.sort_values(by = 'score', ascending = False).reset_index()

,index,1w,1b,2w,2b,3w,3b,mean,count,standard_deviation,95%_confidence_interval_+,95%_confidence_interval_-,+/-range,previous_count,%_of_previous_position,countscore,eloscore,rarityscore,score
0,11789,d4,Nf6,Bg5,e6,e4,d6,-111.884615,26,349.747032,22.553815,-246.323046,134.438431,2603,0.009988,9.805807,8.950769,10.000000,28.756576
1,12365,d4,Nf6,Nf3,e6,e3,g6,-124.157895,38,238.487750,-48.329764,-199.986025,75.828131,5319,0.007144,8.111071,9.932632,10.000000,28.043703
2,15377,d4,d5,c4,dxc4,e3,b5,-117.676471,34,319.882338,-10.152108,-225.200833,107.524362,3122,0.010890,8.574929,9.414118,10.000000,27.989047
3,8006,c4,Nf6,g3,e5,Bg2,h6,-138.054054,37,310.297165,-38.069476,-238.038632,99.984578,778,0.047558,8.219949,10.000000,9.513514,27.733463
4,10047,c4,e6,Nf3,d5,d4,a6,-91.724138,29,248.988510,-1.101567,-182.346709,90.622571,1144,0.025350,9.284767,7.337931,10.000000,26.622698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27934,27934,h4,e5,h5,d5,c3,c5,400.000000,1,NaN,NaN,NaN,NaN,2,0.500000,NaN,NaN,NaN,NaN
27935,27935,h4,e5,h5,d5,c3,h6,206.000000,1,NaN,NaN,NaN,NaN,2,0.500000,NaN,NaN,NaN,NaN
27936,27936,h4,e6,h5,c5,c3,Nc6,-36.000000,1,NaN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN
27937,27937,h4,h5,Nf3,g6,Ng5,Bg7,-117.000000,1,NaN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN


In [46]:
w4 = chess.groupby(['1w','1b','2w','2b','3w','3b','4w']).agg(mean = ('white_perf_diff_adjusted', 'mean'), count = ('white_perf_diff_adjusted', 'count'), standard_deviation = ('white_perf_diff_adjusted', 'std'))
w4['95%_confidence_interval_+'] = w4['mean'] + 1.96*(w4['standard_deviation']/(np.sqrt(w4['count'])))
w4['95%_confidence_interval_-'] = w4['mean'] - 1.96*(w4['standard_deviation']/(np.sqrt(w4['count'])))
w4['+/-range'] = (w4['95%_confidence_interval_+']-w4['95%_confidence_interval_-'])/2
w4 = w4.reset_index()
w4p = w4.groupby(['1w','1b','2w','2b','3w','3b']).agg(previous_count = ('count','sum'))
w4 = w4.merge(w4p, how = 'outer', on = ['1w','1b','2w','2b','3w','3b'])
w4['%_of_previous_position'] = w4['count'] / w4['previous_count']
w4.loc[w4['count'] >= 25, 'countscore'] = 50*(w4['count'])**-0.5
w4.loc[w4['count'] >= 25, 'eloscore'] = (0.08*w4['mean'])
w4.loc[w4['eloscore'] > 10, 'eloscore'] = 10
w4.loc[w4['count'] >= 25, 'rarityscore'] = ((w4['%_of_previous_position']**-1)-2)/2
w4.loc[w4['rarityscore'] < 0, 'rarityscore'] = 0
w4.loc[w4['rarityscore'] > 10, 'rarityscore'] = 10
w4['score'] = w4['countscore']+w4['eloscore']+w4['rarityscore']
w4.sort_values(by = 'score', ascending = False).reset_index()

,index,1w,1b,2w,2b,3w,3b,4w,mean,count,standard_deviation,95%_confidence_interval_+,95%_confidence_interval_-,+/-range,previous_count,%_of_previous_position,countscore,eloscore,rarityscore,score
0,39076,e4,c5,Nc3,d6,f4,Nc6,Bc4,144.960000,25,329.741121,274.218519,15.701481,129.258519,927,0.026969,10.000000,10.000000,10.000000,30.000000
1,24227,d4,Nf6,c4,d6,Nc3,g6,g3,125.620690,29,192.088970,195.533941,55.707438,69.913251,621,0.046699,9.284767,10.000000,9.706897,28.991663
2,33115,d4,e6,c4,b6,a3,Bb7,d5,143.346154,26,262.642562,244.302704,42.389603,100.956551,509,0.051081,9.805807,10.000000,8.788462,28.594268
3,31610,d4,d6,e4,Nf6,Nc3,g6,Be2,106.920000,25,250.193079,204.995687,8.844313,98.075687,1095,0.022831,10.000000,8.553600,10.000000,28.553600
4,31619,d4,d6,e4,Nf6,Nc3,g6,h3,102.677419,31,257.064893,193.171046,12.183793,90.493627,1095,0.028311,8.980265,8.214194,10.000000,27.194459
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54486,54486,h4,e5,h5,d5,c3,c5,d4,400.000000,1,NaN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN
54487,54487,h4,e5,h5,d5,c3,h6,d4,206.000000,1,NaN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN
54488,54488,h4,e6,h5,c5,c3,Nc6,d3,-36.000000,1,NaN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN
54489,54489,h4,h5,Nf3,g6,Ng5,Bg7,e4,-117.000000,1,NaN,NaN,NaN,NaN,1,1.000000,NaN,NaN,NaN,NaN


In [ ]:
b3 = chess.groupby(['1w','1b','2w','2b','3w','3b']).agg(mean = ('white_perf_diff_adjusted', 'mean'), count = ('white_perf_diff_adjusted', 'count'), standard_deviation = ('white_perf_diff_adjusted', 'std'))
b3['95%_confidence_interval_+'] = b3['mean'] + 1.96*(b3['standard_deviation']/(np.sqrt(b3['count'])))
b3['95%_confidence_interval_-'] = b3['mean'] - 1.96*(b3['standard_deviation']/(np.sqrt(b3['count'])))
b3['+/-range'] = (b3['95%_confidence_interval_+']-b3['95%_confidence_interval_-'])/2
b3 = b3.reset_index()
b3p = b3.groupby(['1w','1b','2w','2b','3w']).agg(previous_count = ('count','sum'))
b3 = b3.merge(b3p, how = 'outer', on = ['1w','1b','2w','2b','3w'])
b3['%_of_previous_position'] = b3['count'] / b3['previous_count']
b3.loc[b3['count'] >= 25, 'countscore'] = 50*(b3['count'])**-0.5
b3.loc[b3['count'] >= 25, 'eloscore'] = (-0.08*b3['mean'])
b3.loc[b3['eloscore'] > 10, 'eloscore'] = 10
b3.loc[b3['count'] >= 25, 'rarityscore'] = ((b3['%_of_previous_position']**-1)-2)/2
b3.loc[b3['rarityscore'] < 0, 'rarityscore'] = 0
b3.loc[b3['rarityscore'] > 10, 'rarityscore'] = 10
b3['score'] = b3['countscore']+b3['eloscore']+b3['rarityscore']
b3.sort_values(by = 'score', ascending = False).reset_index().head(50)

In [ ]:
#I am attempting to make a 'scoring' sytem that scores moves based on: 
# 1. count of times move has appeared (less is better).
# 2. Moves elo score perf (scores with low cofidence will be penalized).
# 3. % of previous position this move is played (lower is better).
# 4. (For now I don't see a good way to do this one) Consider win% for must win white and black situations
# (how do I do this while accounting for elo diff?)
# *Scores will only be calculated if count is 25 or higher to prevent a massive number of possibilites.
# I'm currently stuck on #3- how do I compare a move to the previous one? Time will tell. 

In [17]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 2000)